In [1]:
# Import libraries
import numpy as np
import pandas as pd
from pandas.tseries.offsets import CustomBusinessDay

In [44]:
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go

import plotly
plotly.tools.set_credentials_file(username='raphaelrrcoelho', api_key='H2lY25XX3A0IkijhzRmA')

In [3]:
from modules import io, web, data, risk
import pandas_datareader.data as pd_web
import quandl

# from statsmodels import regression
# from scipy.stats import norm
# import statsmodels.api as sm

# from financial_risk import web, data, risk, io, plot, report

In [4]:
br_holidays = io.get_holidays()

In [5]:
start = pd.datetime.today().date() - CustomBusinessDay(900, holidays=br_holidays) # 252 Business Days Equals * 3 (Years)
end = pd.datetime.today().date() - CustomBusinessDay(1, holidays=br_holidays) # Yesterday

date_index = pd.date_range(start, end, freq=CustomBusinessDay(holidays=br_holidays))

In [6]:
tickers = io.tickers(file='database.xlsx')

In [7]:
stocks = web.stocks(tickers, start, end)
stocks = stocks[:, date_index, :]

BIDI11
SMLE3
VALE5


C:\Users\Raphael\Downloads\hedge-fund-master\hedge-fund-master\modules\web.py:16: FutureWarning:


Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.


C:\Users\Raphael\Anaconda3\lib\site-packages\pandas\core\indexing.py:1161: FutureWarning:


Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.


C:\Users\Raphael\Downloads\hedge-fund-master\hedge-fund-master\modules\web.py:17: FutureWarning

In [8]:
risk_free = web.risk_free(start, end, date_index)

C:\Users\Raphael\Anaconda3\lib\site-packages\pandas\core\indexing.py:116: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike



In [9]:
close_prices = stocks.loc[:, :, 'Close'].fillna(method='ffill').fillna(method='bfill').dropna(axis=1)
adj_close_prices = stocks.loc[:, :, 'Adj Close'].fillna(method='ffill').fillna(method='bfill').dropna(axis=1)

In [10]:
adj_close_prices.head()

,ABEV3,BBAS3,BBDC3,BBDC4,BBSE3,BRAP4,BRFS3,BRKM5,BRML3,B3SA3,...,VALE3,MPLU3,VIVT4,WEGE3,BEEF3,AZUL4,VLID3,SBSP3,SUZB3,ibov
2015-04-29,17.110661,21.659424,19.955036,20.133018,27.952488,9.607440,62.865139,10.323692,10.744433,10.701515,...,18.931799,25.731129,40.545261,11.168038,7.803255,22.4,37.523205,16.542849,19.585752,55325.0
2015-04-30,17.201431,22.175917,19.928499,20.548019,28.379923,9.964503,62.170063,10.566898,10.364489,10.885272,...,20.312899,24.660526,40.109123,11.301958,8.416228,22.4,38.764286,16.496510,19.585752,56229.0
2015-05-04,17.555449,22.025965,19.795818,20.650171,28.315407,10.545765,62.507938,11.774544,10.902502,11.130277,...,22.223116,25.195822,39.802200,11.696472,8.688662,22.4,39.484131,17.173052,19.713055,57354.0
2015-05-05,17.764225,22.634094,20.089941,20.735182,28.339602,11.127027,62.266598,11.246198,11.292301,11.182777,...,24.267857,27.718340,38.889538,11.844496,9.311364,22.4,38.557419,17.163786,19.713055,58052.0
2015-05-06,18.009310,22.584114,19.791248,20.223951,28.992849,10.786572,61.590839,11.388768,10.849653,11.182777,...,22.357641,28.370966,38.744152,11.717634,9.019473,22.4,38.681561,17.682777,19.713055,57103.0


In [11]:
simple_returns = data.simple_returns(adj_close_prices)
cc_returns = np.log(1 + simple_returns)

In [12]:
# ewm_cov = risk.ewm_cov(cc_returns, window=75)
# ewm_mean = risk.ewm_mean(cc_returns, window=75)
# ewm_var = risk.ewm_variance(cc_returns, window=75)

In [13]:
# betas = risk.get_betas(ewm_cov).tail(1)
# betas = pd.Series(betas.ix[0, :].values, index=betas.columns, name='beta')
# tickers['beta'] = betas

In [14]:
# variance = pd.Series(ewm_var.tail(1).ix[0, :].values, index=ewm_var.tail(1).columns, name='daily_volatility')
#  tickers['daily_volatility'] = variance.head(-1) ** 0.5

In [15]:
# mean = pd.Series(ewm_mean.tail(1).ix[0, :].values, index=ewm_mean.tail(1).columns, name='average_return')

In [16]:
# tickers['daily_value_at_risk_95'] = risk.parametric_value_at_risk(mean.head(-1), variance.head(-1) ** 0.5, 0.05)
# tickers['daily_value_at_risk_99'] = risk.parametric_value_at_risk(mean.head(-1), variance.head(-1) ** 0.5, 0.01)

In [17]:
# sharpe_ratio = risk.sharpe_ratio(mean, variance)
# tickers['sharpe_ratio'] = sharpe_ratio

In [18]:
io.save_data(tickers, adj_close_prices, close_prices, simple_returns, cc_returns, file='database.xlsx')

### TOP 5 Analysis

In [74]:
top5 = pd.read_excel("portfolio.xlsx", sheet_name='Portfolio')
top5['start'] = pd.to_datetime(top5['start'])
top5.head()

,start,ticker_1,ticker_2,ticker_3,ticker_4,ticker_5
0,2015-12-30,BBDC4,BRFS3,CPFE3,VIVT4,WEGE3
1,2016-01-29,BBDC4,BRFS3,CPFE3,VIVT4,WEGE3
2,2016-02-01,BBDC4,BRFS3,CPFE3,SUZB3,VIVT4
3,2016-02-24,BBDC4,CPFE3,MPLU3,SUZB3,VIVT4
4,2016-02-29,BBDC4,CPFE3,MPLU3,SUZB3,VIVT4


In [75]:
top5['end'] = top5['start'].shift(-1) - CustomBusinessDay(1, holidays=br_holidays)

top5 = top5.loc[:, ['start', 'end', 'ticker_1', 'ticker_2', 'ticker_3', 'ticker_4', 'ticker_5']]

top5.iloc[-1, 1] = pd.datetime.today().date() - CustomBusinessDay(1, holidays=br_holidays) # Yesterday

top5.tail()

C:\Users\Raphael\Anaconda3\lib\site-packages\pandas\core\indexes\datetimes.py:962: PerformanceWarning:

Non-vectorized DateOffset being applied to Series or DatetimeIndex



,start,end,ticker_1,ticker_2,ticker_3,ticker_4,ticker_5
84,2018-08-06,2018-08-15,ESTC3,BEEF3,GGBR4,VIVT4,VLID3
85,2018-08-16,2018-08-24,ESTC3,BEEF3,GGBR4,VIVT4,PETR4
86,2018-08-27,2018-09-21,ESTC3,BEEF3,RAPT4,VIVT4,PETR4
87,2018-09-24,2018-11-16,ESTC3,BEEF3,RAPT4,BBDC4,PETR4
88,2018-11-19,2018-12-13,PETR4,CYRE3,ESTC3,AZUL4,BEEF3


In [76]:
top5 = top5.set_index(['start', 'end'])

top5.head()

,,ticker_1,ticker_2,ticker_3,ticker_4,ticker_5
start,end,,,,,
2015-12-30,2016-01-28,BBDC4,BRFS3,CPFE3,VIVT4,WEGE3
2016-01-29,2016-01-29,BBDC4,BRFS3,CPFE3,VIVT4,WEGE3
2016-02-01,2016-02-23,BBDC4,BRFS3,CPFE3,SUZB3,VIVT4
2016-02-24,2016-02-26,BBDC4,CPFE3,MPLU3,SUZB3,VIVT4
2016-02-29,2016-03-08,BBDC4,CPFE3,MPLU3,SUZB3,VIVT4


In [105]:
starts = top5.index.get_level_values(0)

allocations = [ row.values for _, row in top5.iterrows() ]
allocations = [ ' '.join(i) for i in allocations ]

allocations = [ date + ": " + allocation for date, allocation in zip(starts.strftime("%Y-%m-%d").tolist(), allocations) ]

In [107]:
returns_cc = []

for _, row in top5.iterrows():
    start = row.name[0]
    end = row.name[1]
    allocation = row.values

    returns_cc.append( cc_returns.loc[start:end, allocation].sum().sum() / 5 )

returns = np.exp(returns_cc) - 1
cum_returns = np.exp(np.cumsum(returns_cc)) - 1


C:\Users\Raphael\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike



In [108]:
data = [
    go.Scatter(
        x = starts,
        y = returns,
        text = allocations,
        hoverinfo = 'text',
        marker = dict(
            color = 'green'
        ),
        showlegend = False
    )
]

py.iplot(data, filename = "top5_returns")

In [109]:
data = [
    go.Scatter(
        x = starts,
        y = cum_returns,
        text = allocations,
        hoverinfo = 'text',
        marker = dict(
            color = 'green'
        ),
        showlegend = False
    )
]

layout = dict(
    title = "Top 5 Cumulative Returns",
    xaxis = dict(
        range = [starts[0], starts[1]])
)

fig = dict(data=data, layout=layout)
py.iplot(data, filename = "top5_cumulative_returns")

In [110]:
yesterday = pd.datetime.today().date() - CustomBusinessDay(1, holidays=br_holidays)

In [114]:
adj_close_prices.loc[starts[0]:yesterday, :].index

DatetimeIndex(['2015-12-30', '2016-01-04', '2016-01-05', '2016-01-06',
               '2016-01-07', '2016-01-08', '2016-01-11', '2016-01-12',
               '2016-01-13', '2016-01-14',
               ...
               '2018-11-30', '2018-12-03', '2018-12-04', '2018-12-05',
               '2018-12-06', '2018-12-07', '2018-12-10', '2018-12-11',
               '2018-12-12', '2018-12-13'],
              dtype='datetime64[ns]', length=734, freq='C')

In [117]:
cum_returns[-3:]

array([1.59685593, 2.18295342, 2.08189018])

In [120]:
(1 + 2.08189018)**(252./734) - 1

0.47171184348008177